In [1]:
%%file t.cpp
extern "C" const char* echo(const char* t) { return t; } 

Writing t.cpp


In [26]:
%%file t.py
from ctypes import *                                                                                                                                                                 
c_object_p = POINTER(c_void_p)

import sys 
if sys.version_info[0] == 3:  
# Python 3 strings are unicode, translate them to/from utf8 for C-interop.
    class c_interop_string(c_char_p):
        def __init__(self, p=None):
            if p is None:
                p = ""
            if type(p) == str:
                p = p.encode("utf8")
            super(c_char_p, self).__init__(p)

        @property
        def value(self):
            return super(c_char_p, self).value.decode("utf8")

        @classmethod
        def from_param(cls, param):
            return cls(param)

    def _utf8_to_python_string(x, *args):
        return x.decode("utf8")

elif sys.version_info[0] == 2:  
# Python 2 strings are utf8 byte strings, no translation is needed for C-interop.
    c_interop_string = c_char_p

    def _utf8_to_python_string(x, *args):
        return x

else:
    raise Exception("Only Python versions 2 and 3 are supported.")


def echo(s):
    return lib.echo(s)

lib = cdll.LoadLibrary("libt_c.dylib")

methodList = [ 
    ("echo",
    [c_interop_string],
    c_char_p,
    _utf8_to_python_string)
]

def register_method(lib, item):
    func = getattr(lib, item[0])

    if len(item) >= 2:
        func.argtypes = item[1]

    if len(item) >= 3:
        func.restype = item[2]

    if len(item) >= 4:
        func.errcheck = item[3]

for m in methodList:
    register_method(lib,m)

Overwriting t.py


In [27]:
%%file CMakeLists.txt
cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
add_library(t_c SHARED t.cpp)

Overwriting CMakeLists.txt


In [28]:
%%sh
cmake .
cmake --build .

-- Configuring done
-- Generating done
-- Build files have been written to: /Users/jon/Google Drive/iPythonNotebooks
[100%] Built target t_c


In [29]:
%%script python2
import t
assert(t.echo('hello') + ' world' == 'hello world')

In [30]:
%%script python3
import t
assert(t.echo('hello') + ' world' == 'hello world')